# Frontend Application interface to the Stabilization Models

At some point this could be converted to AWS CloudFormation but at this point understanding the infrastructure at a basic AWS SDK point is highly disirable.


## Frontend Setup


Setup the buckets associated with the application.  All the files exist either as code stored in the [project repository](https://github.com/mgrandau/hand-tracking-stabilization) or data files in the [application bucket](s3://mgrandau-springboard-capstone/frontend-application/)

The application is composed of 3 S3 buckets. One bucket contains the application and the canned input videos used for testing. A second bucket is used as an input bucke to the flow. The 3rd and last bucket contains the output of the stabilization python.


In [2]:
import logging
import boto3
from botocore.exceptions import ClientError

def create_bucket(bucket_name, region='us-west-2'):
    try:
        s3_client = boto3.client('s3', region_name=region)
        location = {'LocationConstraint': region}
        s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
        
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [17]:
create_bucket("mgrandau-springboard-capstone-app")
create_bucket("mgrandau-springboard-capstone-input")
create_bucket("mgrandau-springboard-capstone-output")

True

Load the application files.

In [18]:
s3 = boto3.resource('s3')

def copy_to_application(filename):
    copy_source = {
        'Bucket': 'mgrandau-springboard-capstone',
        'Key': f'frontend-application/{filename}'
    }
    s3.meta.client.copy(copy_source, 'mgrandau-springboard-capstone-app', filename)
    object_acl = s3.ObjectAcl('mgrandau-springboard-capstone-app',filename)
    response = object_acl.put(ACL='public-read')
    
copy_to_application('photoExample.html')
copy_to_application('photoExample.js')
copy_to_application('PhotoViewer.html')
copy_to_application('PhotoViewer.js')

This is the [photoExample](https://mgrandau-springboard-capstone-app.s3.us-west-2.amazonaws.com/photoExample.html)

## Frontend Teardown

Cleanup the application buckets

In [8]:
def delete_bucket(bucket_name):
    try:
        import boto3

        s3_client = boto3.resource('s3')
        bucket = s3_client.Bucket(bucket_name)
        bucket.objects.all().delete()
        bucket.delete()
        
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [16]:
delete_bucket("mgrandau-springboard-capstone-app")
delete_bucket("mgrandau-springboard-capstone-input")
delete_bucket("mgrandau-springboard-capstone-output")

True